In [1]:
%matplotlib inline
import pandas
import glob

In [30]:


images = pandas.read_csv('psi/data/IAPS.csv', sep=';', usecols=['IAPS', 'ValenceMean', 'ArousalMean'])

sounds = pandas.read_csv('psi/data/IADS2.csv', sep=';', usecols=['Number', 'ValenceMean', 'ArousalMean'])


files = glob.glob('psi/data/procedura/*.txt')

li = []

for filename in files:
    df = pandas.read_csv(filename, sep='\t', header=None, names=['ID', 'nn', 'Condition', 'SpecificCondition', 'SoundNumber', 'ImageIAPS', 'Widget', 'Response', 'Delay', 'ApperanceTimestamp'])
    li.append(df)
                         
studyData = pandas.concat(li, axis=0, ignore_index=True)
                         
studyData = studyData[studyData.Condition != 'con']
studyData = studyData[studyData.Widget != 'emoscale1']

studyData
                         


,ID,nn,Condition,SpecificCondition,SoundNumber,ImageIAPS,Widget,Response,Delay,ApperanceTimestamp
6,1107,6,inc,p+s-,422,4800,emospace1,"[0.0, -0.8871154785156252]",2.803023644906716,1.555651e+09
7,1107,7,inc,p-s+,355,3150,emospace1,"[-0.889205932617187, 0.0]",2.419283322857609,1.555651e+09
8,1107,8,inc,p+s-,277,4647,emospace1,"[-0.6720916748046881, -0.6839477539062505]",2.686227278539718,1.555651e+09
9,1107,9,inc,p+s-,719,5215,emospace1,"[-0.6523760986328123, 0.6662744140625002]",2.5193468970223876,1.555651e+09
10,1107,10,inc,p-s+,311,8230,emospace1,"[-0.848992919921875, 0.0]",2.3358856049239876,1.555651e+09
14,1107,14,inc,p-s+,808,3010,emospace1,"[0.8944860839843755, -0.016984252929687502]",2.853136775703206,1.555651e+09
18,1107,18,inc,p+s-,288,4668,emospace1,"[0.6574371337890625, -0.6201428222656251]",2.969861103356834,1.555651e+09
19,1107,19,inc,p+s-,290,4697,emospace1,"[-0.637518310546875, -0.62601806640625]",2.13605141402968,1.555651e+09
20,1107,20,inc,p-s+,815,9622,emospace1,"[0.6604333496093752, 0.6413775634765618]",1.968839354269221,1.555651e+09
22,1107,22,inc,p-s+,716,9184,emospace1,"[-0.8678228759765627, 0.0]",1.8683393455639816,1.555651e+09


In [ ]:
studyDataArray = [pandas.read_csv(filename, sep="\t", header=None, lineterminator='\n') for filename in glob.glob("psi/data/procedura/*.txt")]

studyData = pandas.concat(studyDataArray, axis=0)